In [1]:
import transformers, huggingface_hub, datasets 
from transformers import pipeline
from json import loads, dumps

import openai, logging, os, socket, csv, json, random, uuid # type: ignore
from datetime import datetime, timedelta
from io import BytesIO, StringIO

/Users/skingsle/opt/anaconda3/envs/new_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
def get_ai_response(message):
    openai_api_key = os.getenv("OPENAI_API_KEY")
    if not openai_api_key:
        raise RuntimeError("No OpenAI API key found in the environment variables. Please set 'OPENAI_API_KEY' in the .env file.")
    
    openai.api_key = openai_api_key
    
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Respond to this {message}"},
            {"role": "user", "content": message}])

    response=response.choices[0].message.content
    
    timestamp_aiResponse_received = datetime.now().isoformat()
    session[' timestamp_aiResponse_received'] =  timestamp_aiResponse_received
    
    return response

def get_llama_response(message):
    from dotenv import load_dotenv
    load_dotenv()
    
    HF_TOKEN = os.getenv("HUGGINGFACE_TOKEN")
    if not HF_TOKEN:
        raise RuntimeError("No Hugging Face API token found in the environment variables. Please set 'HUGGINGFACE_TOKEN' in the .env file.")
    
    #from huggingface_hub import login
    #login(token=HF_TOKEN)

    pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-1B", token=HF_TOKEN)  

    if isinstance(message, str):
        messages = [message]  # Convert the single prompt to a list

    responses = []
    system_prompt = "Provide a response to the user."

    for prompt in messages:
        # Generate response from the LLM
        try:
            # Assuming the model can handle single string inputs as well
            outputs = pipe(f"{system_prompt} {prompt}", max_length=150, num_return_sequences=1, truncation=True)
            response = outputs[0]["generated_text"]
            responses.append(response)
        except Exception as e:
            responses.append(f"Error generating response: {e}")

    return responses if len(responses) > 1 else responses[0]  # Return a single response or a list

In [14]:
get_llama_response(message="hello, how are you?")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'Provide a response to the user. hello, how are you?'

In [ ]:
import sqlite3

def fetch_all_records_from_prompts_responses(db_path):
    """
    Fetch all records from the prompts_responses table.

    :param db_path: The path to the SQLite database file.
    :return: A list of tuples, each representing a row in the table.
    """
    try:
        # Connect to the SQLite database
        connection = sqlite3.connect(db_path)
        cursor = connection.cursor()

        # Define the SQL query to retrieve all records
        query = "SELECT * FROM prompts_responses;"

        # Execute the query
        cursor.execute(query)

        # Fetch all records from the table
        rows = cursor.fetchall()

        # Close the cursor and connection
        cursor.close()
        connection.close()

        return rows

    except sqlite3.Error as e:
        print(f"An error occurred while fetching records: {e}")
        return []


In [23]:
def get_postgres_connection_pool():
    try:
        # Use the connection string directly
        connection_string = os.environ['PGCONNECTIONSTRING']

        # Create a connection pool
        pg_pool = psycopg2.pool.SimpleConnectionPool(1, 10, dsn=connection_string)  # Adjust minconn and maxconn as needed

        if pg_pool:
            print("Connection pool created successfully")

        # Get a connection from the pool
        connection = pg_pool.getconn()
        if connection:
            print("Successfully received connection from pool")

        return pg_pool, connection

    except (Exception, psycopg2.DatabaseError) as error:
        print("Error while connecting to PostgreSQL", error)

In [22]:
#!pip install psycopg2-binary
import psycopg2 # type: ignore
import psycopg2.extras # type: ignore
from psycopg2 import pool # type: ignore

def get_user_id_genailab(user_id):
  pg_pool, connection = get_postgres_connection_pool()
  cursor = connection.cursor()
  cursor.execute('''SELECT user_id FROM genailab_users WHERE user_id= %s;''', (user_id,))
  user_ids = cursor.fetchall()
  pg_pool.putconn(connection)
  return user_id
  

Connection pool created successfully
Successfully received connection from pool


InvalidTextRepresentation: invalid input syntax for type integer: "test"
LINE 1: SELECT user_id FROM genailab_users WHERE user_id= 'test';
                                                          ^


In [28]:
pg_pool, connection = get_postgres_connection_pool()
cursor = connection.cursor()
cursor.execute('''SELECT * FROM prompts_responses;''')

Connection pool created successfully
Successfully received connection from pool


In [29]:
cursor.fetchall()

[]